Subclassing Built-In Types Is Tricky Examples

In [2]:
class DoppelDict(dict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value]*2)

dd = DoppelDict(one = 1)

print(dd)

dd['two'] = 2

print(dd)

dd.update(three=3)
print(dd)


{'one': 1}
{'one': 1, 'two': [2, 2]}
{'one': 1, 'two': [2, 2], 'three': 3}


In [3]:
class AnswerDict(dict):
    def __getitem__(self, key):
        return 42

ad = AnswerDict(a='foo')
print(ad['a'])

d = {}
d.update(ad)
print(d['a'])

42
foo


In [4]:
import collections

class DoppelDict2(collections.UserDict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value] * 2)

Multiple Inheritance and Method Resolution Order

In [11]:
class Root:
    def ping(self):
        print(f'{self}.ping() in Root')
    
    def pong(self):
        print(f'{self}.pong() in Root')
    
    def __repr__(self):
        cls_name = type(self).__name__
        return f'<instance of {cls_name}>'

class A(Root):
    def ping(self):
        print(f'{self}.ping() in A')
        super().ping()
    
    def pong(self):
        print(f'{self}.pong() in A')
        super().pong()

class B(Root):
    def ping(self):
        print(f'{self}.ping() in B')
        super().ping()
    
    def pong(self):
        print(f'{self}.pong() in A')

class Leaf(A,B):
    def ping(self):
        print(f'{self}.ping() in Leaf')
        super().ping()

leaf1 = Leaf()
leaf1.ping()
leaf1.pong()


<instance of Leaf>.ping() in Leaf
<instance of Leaf>.ping() in A
<instance of Leaf>.ping() in B
<instance of Leaf>.ping() in Root
<instance of Leaf>.pong() in A
<instance of Leaf>.pong() in A


In [18]:
class U():
    def ping(self):
        print(f'{self}.ping() in U')
        super().ping()
    
class LeafUA(U, A):
    def ping(self):
        print(f'{self}.ping() in LeafUA')
        super().ping()

leaf2 = LeafUA()
leaf2.ping()
leaf2.pong()


<instance of LeafUA>.ping() in LeafUA
<instance of LeafUA>.ping() in U
<instance of LeafUA>.ping() in A
<instance of LeafUA>.ping() in Root
<instance of LeafUA>.pong() in A
<instance of LeafUA>.pong() in Root


In [20]:
import tkinter
def print_mro(cls):
    print(', '.join(c.__name__ for c in cls.__mro__))
print_mro(tkinter.Text)

Text, Widget, BaseWidget, Misc, Pack, Place, Grid, XView, YView, object


Mixin Classes Examples

In [3]:
import collections

def _upper(key):
    try:
        return key.upper()
    except AttributeError:
        return key

class UpperCaseMixin:
    def __setitem__(self, key, item):
        super().__setitem__(_upper(key), item)

    def __getitem__(self, key):
        return super().__getitem__(_upper(key))

    def get(self, key, default=None):
        return super().get(_upper(key), default)

    def __contains__(self, key):
        return super().__contains__(_upper(key))

class UpperDict(UpperCaseMixin, collections.UserDict):
    pass

class UpperCounter(UpperCaseMixin, collections.Counter):
    """Specialized 'Counter' that upper-cases string keys"""

d = UpperDict([('a', 'letter A'), (2, 'digit two')])
print(list(d.keys()))
d['b'] = 'letter B'
print('b' in d)
print(d['a'], d.get('B'))
print(list(d.keys()))

c = UpperCounter('BaNanA')
print(c.most_common())

['A', 2]
True
letter A letter B
['A', 2, 'B']
[('A', 3), ('N', 2), ('B', 1)]
